In [1]:
import pandas as pd

import matplotlib.pyplot as plt


games_df = pd.read_csv('games.csv')
games_df = games_df.loc[(games_df['Year_of_Release']>=2000)].dropna()
#games_df.loc[games_df['Genre'].isin(['Sports']) & games_df['Rating'].isin(['E'])]

In [71]:
g=games_df.groupby(['Name','Year_of_Release']).agg(['count'])
gff = g.where(g>1).dropna().reset_index()
gff.loc[gff['Name']=='Grand Theft Auto V']

,Name,Year_of_Release,Platform,Genre,Critic_Score,User_Score,Rating
,,,count,count,count,count,count
477,Grand Theft Auto V,2013.0,2.0,2.0,2.0,2.0,2.0
478,Grand Theft Auto V,2014.0,2.0,2.0,2.0,2.0,2.0


In [84]:
print(games_df.Genre.value_counts(),'\n',games_df.Rating.value_counts())

Action          1821
Sports          1144
Shooter          912
Role-Playing     707
Racing           706
Misc             502
Platform         483
Fighting         394
Simulation       335
Adventure        295
Strategy         279
Puzzle           214
Name: Genre, dtype: int64 
 E       2709
T       2543
M       1443
E10+    1095
AO         1
RP         1
Name: Rating, dtype: int64


In [171]:
games_df.loc[(games_df.Genre.isin(list(['Racing']))) & (games_df.Rating.isin(list(['T'])))]

,Name,Platform,Year_of_Release,Genre,Critic_Score,User_Score,Rating
253,Need for Speed: Most Wanted,PS2,2005.0,Racing,82.0,9.1,T
317,MotorStorm,PS3,2006.0,Racing,84.0,7.4,T
355,The Simpsons: Road Rage,PS2,2001.0,Racing,64.0,7.7,T
562,Burnout 3: Takedown,PS2,2004.0,Racing,93.0,9.1,T
579,Midnight Club: Street Racing,PS2,2000.0,Racing,78.0,7.8,T
...,...,...,...,...,...,...,...
15834,Top Gear RPM Tuning,XB,2005.0,Racing,45.0,7.5,T
16242,Furious Karting,XB,2003.0,Racing,63.0,tbd,T
16421,FlatOut: Ultimate Carnage,PC,2008.0,Racing,79.0,7.7,T
16459,Powerdrome,XB,2004.0,Racing,69.0,7,T


In [219]:
games_df.groupby(["Platform", "Year_of_Release"],as_index=False).size().reset_index()

,index,Platform,Year_of_Release,size
0,0,3DS,2011.0,61
1,1,3DS,2012.0,33
2,2,3DS,2013.0,20
3,3,3DS,2014.0,22
4,4,3DS,2015.0,19
...,...,...,...,...
120,120,XB,2007.0,3
121,121,XOne,2013.0,14
122,122,XOne,2014.0,43
123,123,XOne,2015.0,47
